Testando se o ambiente está funcionando

In [1]:
print('teste')

teste


Importando bibliotecas para explorar os diretórios do computador

In [2]:
from os import listdir
from os.path import isfile, join

Verificando todos os arquivos obtidos da fonta de dados

In [3]:
path = "E://dump"

dump_files = listdir(path)

In [4]:
import pandas as pd
import numpy as np

Tendo certeza que é possível ler os dados do GHtorrent.

In [6]:
#pd.read_csv(join(path, 'project_languages.csv'))

Acredito que será possível partir dos pullrequest para observar quais repositórios possuem contribuições ou não. Além disso, após pesquisar (https://levelup.gitconnected.com/how-to-mine-github-data-in-2022-e9c70b3f61d3) parece que os dados de 2015 possuem uma estrutura diferente. Então irei tentar filtrar eles fora do meu dataset

In [7]:
#df_pullrequests = pd.read_csv(join(path, 'pull_requests.csv'))

In [8]:
#df_pullrequests

Um artigo que mostra a estrutura dos arquivos do GHtorrent https://gousios.org/pub/ghtorrent-dataset-toolsuite.pdf
misturando com esse aqui também https://github.com/ghtorrent/ghtorrent.org/blob/master/files/schema.pdf
Então irei adicionar o nome nas colunas, acabei retirando a coluna pullrequest_id pois ela não fazia sentido no contexto

In [9]:
#df_pullrequests.columns = ["id", "head_repo_id", "base_repo_id", "head_commit_id", "base_commit_id", "intra_branch_id","merged"]

Como os pull requests não possuem data de criação tentarei filtrar através dos commits que eles estão ligados.

In [10]:
# # Função para calcular o número total de chunks
# def get_total_chunks(filename, chunksize):
#     # Contar o número total de linhas no arquivo
#     total_lines = sum(1 for _ in open(filename)) - 1  # Subtrai 1 por causa do cabeçalho
#     # Calcular o número total de chunks
#     total_chunks = (total_lines // chunksize) + (1 if total_lines % chunksize != 0 else 0)
#     return total_chunks

# # Exemplo de uso
# filename = join(path, 'commits.csv')
# chunksize = 5000000
# total_chunks = get_total_chunks(filename, chunksize)
# print(f"Total de chunks: {total_chunks}")

In [11]:
#df_commits_19 = pd.DataFrame({},columns=["id", "sha", "author_id", "commiter_id", "project_id", "created_at"])



In [12]:
#def checkDate(df):
#   return pd.to_datetime(df.iloc[0]['created_at']) > pd.to_datetime('2019-01-01') or pd.to_datetime(df.iloc[-1]['created_at']) > pd.to_datetime('2019-01-01')




In [13]:
# for chunk in df_commits:
#     chunk.map(lambda row: chechRowDate(row))

In [14]:
# i=0

# for chunk in df_commits:
#     if checkDate(chunk):
#         chunk.to_csv(f'E:/dump/commits_greater_than_2k19_{i}')

#     i += 1

# Filtrando os repositórios Open Source

Devido a grande quantidade de commits, não foi possível os utilizar como filtro de projetos open-source. Nesse caso, iremos utilizar a relação project_members para encontrar quais projetos são relevantes, ou seja, que possuem contribuintes além do dono.

In [7]:
df_project_members = pd.read_csv(join(path, 'project_members.csv'), names=["repo_id", "user_id", "created_at"])

In [8]:
df_project_members

,repo_id,user_id,created_at
0,1,1,2009-12-08 10:17:27
1,1,2,2009-12-08 10:17:27
2,1,4,2009-12-08 10:17:27
3,3,24,2011-02-19 01:58:42
4,3,5465,2011-02-19 01:58:42
...,...,...,...
15525047,207191305,221488,2021-03-06 22:46:40
15525048,207191367,54196484,2021-03-06 22:51:54
15525049,207191455,54204581,2021-03-06 22:50:39
15525050,207191498,3518100,2021-03-06 22:45:27


In [9]:
df_count_members = df_project_members.groupby("repo_id", as_index=False)["user_id"].count()

In [11]:
df_open_source_members = df_count_members[df_count_members["user_id"]>10]

In [12]:
df_open_source_members

,repo_id,user_id
1,3,19
4,8,16
14,19,14
26,34,38
27,37,42
...,...,...
9544854,206859445,12
9545989,206918002,12
9546379,206955864,17
9547045,206997704,19


Devivo a datação dos dados iremos filtrar somente os membros a partir de 2019

In [13]:
df_project_members = df_project_members.loc[
    (pd.to_datetime(df_project_members['created_at']) > pd.to_datetime("01-01-2019")) & 
    (df_project_members['repo_id'].isin(df_open_source_members['repo_id']))
    ]

In [14]:
#transforma o campo de data em data

df_project_members['created_at'] = pd.to_datetime(df_project_members['created_at'])

C:\Users\gusge\AppData\Local\Temp\ipykernel_21700\3971203305.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_project_members['created_at'] = pd.to_datetime(df_project_members['created_at'])


In [15]:
df_project_members

,repo_id,user_id,created_at
66,19,9245591,2020-06-17 19:28:17
68,19,33292409,2020-06-05 14:53:56
1475,346,32145570,2019-01-29 14:55:47
2199,499,87499,2019-01-10 18:12:44
2201,499,249763,2019-02-27 17:41:17
...,...,...,...
15524292,207159748,61113108,2021-03-06 17:14:58
15524293,207159748,61908309,2021-03-06 17:13:48
15524294,207159748,68016435,2021-03-06 16:52:38
15524295,207159748,68016479,2021-03-06 16:54:11


# Dataframe alvo

A partir do estudo Recommending GitHub Projects for Developer Onboarding podemos extrair alguns atributos a serem extraídos do dataset para que seja estudado o quão receptivo um repositório é, são eles:

- Crescimento do Projeto: Quantidade de commits do projeto no momento de iniciação
- Data de criação do projeto: Projetos mais novos tendem a atrair mais desenvolvedores
- Momento de primeira contribuição: quando o projeto começa a receber contribuidores
- Momento da última contribuição: se a equipe do projeto se manter estável significa que existe menos oportunidades de onboarding
- Momento do primeiro commit: primeiro commit do projeto, a partir de quando passa a aceitar contribuições
- Momento do último commit: se o projeto receber muitos commits, mas nao tiver tanto onboarding significa q esse projeto necessita de mais contribuintes

In [37]:
#objetivo final

df_onboardings = pd.DataFrame({}, columns=["repo_id", "user_id", "onboarding_date", "project_created_at", "total_commits","first_onboarding", "last_onboarding", "first_commit", "last_commit"])

df_onboardings

,repo_id,user_id,onboarding_date,project_created_at,total_commits,first_onboarding,last_onboarding,first_commit,last_commit


In [17]:
chunks_commits = pd.read_csv(join(path, 'commits.csv'), names=["id", "sha", "author_id", "commiter_id", "project_id", "created_at"], chunksize=5000000)

Agora irei filtrar somente os commits dos projetos selecionados como open source após o ano de 2019

# Pegando os commits referentes aos repositórios classificados como Open Source

In [26]:

#i=0
#for chunk in chunks_commits:
#        df_commit = chunk[chunk['project_id'].isin(df_project_members['repo_id'])]
#        df_commit.to_csv(f'E:/dump/open_source/open_source_commits_{i}.csv')
#        i += 1


In [18]:
dump_open_source_files = listdir(join(path, 'open_source'))


# Descobrindo usuários iniciantes

Para determinar quais contribuições são referentes a iniciantes teremos que descobrir qual foi o primeiro commit de cada usuário e qual foi o repositório dele

testando um método de filtrar os primeiros commits em repositórios open source de cada usuário sem q o ID e o repo_id sejam abstraídos

In [19]:
def get_min_date_row(group):
    min_row = group.loc[group['created_at'].idxmin()]
    return min_row



In [32]:
# first_user_commit_by_chunks2 = []

# i=0
# for file in dump_open_source_files:
#     print("lendo arquivo: ", i)
#     df_commit = pd.read_csv(join(path,'open_source',file))
#     df_commit["created_at"] = pd.to_datetime(df_commit["created_at"])
#     first_user_commit_by_chunks2.append(df_commit.groupby('author_id').apply(get_min_date_row).reset_index(drop=True))
#     i += 1

In [33]:
# df_first_user_commit2 = pd.concat(first_user_commit_by_chunks2)

# df_first_user_commit2

In [34]:
# df_first_user_commit2.drop('Unnamed: 0', axis=1)

# df_first_user_commit2.to_csv('first_user_commit.csv')

In [35]:
#df_first_user_commit2 = pd.read_csv('first_user_commit.csv')

In [36]:
#df_first_user_contrib = df_first_user_commit2.groupby('author_id').apply(get_min_date_row).reset_index(drop=True)



,Unnamed: 0.1,Unnamed: 0,id,sha,author_id,commiter_id,project_id,created_at
0,0,1811535861,1883483709,b8ceafa8f487426af00dac5c50639560559df343,1,1,5049,2007-10-14 12:58:15
1,0,20724408,22521551,b44d3b0015b271c84aca4bc9f0168968cb850a0c,2,2,1724197,2008-04-07 00:09:13
2,0,76288056,79129981,31b817bf40eb9c9439e0f40823ba29421af889ed,3,121679,45316,2010-12-03 21:46:12
3,1,21950245,23772038,57423bcd43b6038b2673bc4451749cefb35728db,4,4,1867516,2008-06-12 17:21:44
4,3,11511319,12856279,88345134f57cfa026fda2bddd6548f73c2641467,5,5,913123,2008-05-11 13:16:20
...,...,...,...,...,...,...,...,...
6901775,19973,2046565013,2132218602,71b525eec9a1bd4f02ed41aaae8ed067add9ee0a,67994955,67994955,205036147,2021-03-05 01:03:21
6901776,19974,2046568897,2132222532,8277df669a3a07058784177fe0cbb7bf863bca48,67995186,67995186,206941146,2021-03-05 11:28:14
6901777,19975,2046569344,2132222983,9891835b6e35014a62bb3e8bc9f58bd1daeb6cc5,67995216,67995216,204201846,2021-03-05 12:22:52
6901778,19976,2046570260,2132223910,9a5ee877d0dd347be3e3fac425aef4fc05485bda,67995268,67995268,206861821,2021-03-05 12:23:22


In [ ]:
#df_first_user_contrib = df_first_user_contrib.drop('Unnamed: 0', axis=1)
#df_first_user_contrib = df_first_user_contrib.drop('Unnamed: 0.1', axis=1)

In [41]:
#df_first_user_contrib.to_csv('first_user_contrib.csv')

In [20]:
df_first_user_contrib = pd.read_csv("first_user_contrib.csv")

Agora iremos filtrar somente as primeiras contribuições de cada pessoa, já que estamos focando em pessoas iniciantes

In [30]:
df_first_user_contrib = df_first_user_contrib.drop('Unnamed: 0', axis=1)
df_first_user_contrib = df_first_user_contrib[pd.to_datetime(df_first_user_contrib['created_at']) > pd.to_datetime('01-01-2019')]

In [31]:
df_first_user_contrib

,id,sha,author_id,commiter_id,project_id,created_at
391,1368679517,b31d4edd720ccbe73c41524443cff2aad99e9e63,522,4880649,94155879,2019-04-21 09:52:05
770,1361915567,8a17770a7f55378cdfdfc834c78b52979577e659,1002,28994,19962242,2019-04-12 10:05:09
1071,1265257924,55251aa89099358c040d38f3af48e3699d67bab2,1409,1006514,67114901,2019-01-10 17:24:38
1078,1884682839,2371e8d2041c2d647d23edf5e073b4a0521cf059,1417,36421502,21624052,2019-07-18 04:43:05
2074,1808726819,2a24064bb220d90277d975b26411a626c47f822c,2686,2686,114639466,2020-08-27 19:59:39
...,...,...,...,...,...,...
6901775,2132218602,71b525eec9a1bd4f02ed41aaae8ed067add9ee0a,67994955,67994955,205036147,2021-03-05 01:03:21
6901776,2132222532,8277df669a3a07058784177fe0cbb7bf863bca48,67995186,67995186,206941146,2021-03-05 11:28:14
6901777,2132222983,9891835b6e35014a62bb3e8bc9f58bd1daeb6cc5,67995216,67995216,204201846,2021-03-05 12:22:52
6901778,2132223910,9a5ee877d0dd347be3e3fac425aef4fc05485bda,67995268,67995268,206861821,2021-03-05 12:23:22


In [24]:
df_project_members

,repo_id,user_id,created_at
66,19,9245591,2020-06-17 19:28:17
68,19,33292409,2020-06-05 14:53:56
1475,346,32145570,2019-01-29 14:55:47
2199,499,87499,2019-01-10 18:12:44
2201,499,249763,2019-02-27 17:41:17
...,...,...,...
15524292,207159748,61113108,2021-03-06 17:14:58
15524293,207159748,61908309,2021-03-06 17:13:48
15524294,207159748,68016435,2021-03-06 16:52:38
15524295,207159748,68016479,2021-03-06 16:54:11


In [34]:

df_contrib_x_commits = df_project_members.merge(df_first_user_contrib, left_on=['repo_id', 'user_id'], right_on=['project_id', 'author_id'],  how='inner', suffixes=['_members', '_commits'])


In [38]:
df_onboardings['repo_id'] = df_contrib_x_commits['repo_id']
df_onboardings['user_id'] = df_contrib_x_commits['author_id']
df_onboardings['onboarding_date'] = df_contrib_x_commits['created_at_members']

df_onboardings

,repo_id,user_id,onboarding_date,project_created_at,total_commits,first_onboarding,last_onboarding,first_commit,last_commit
0,19,9245591,2020-06-17 19:28:17,NaN,NaN,NaN,NaN,NaN,NaN
1,19,33292409,2020-06-05 14:53:56,NaN,NaN,NaN,NaN,NaN,NaN
2,4567,32547120,2020-01-04 17:08:41,NaN,NaN,NaN,NaN,NaN,NaN
3,6830,44244864,2019-04-19 01:58:41,NaN,NaN,NaN,NaN,NaN,NaN
4,8373,30552107,2020-06-05 20:09:57,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
16626,206997704,66799493,2021-03-05 18:06:24,NaN,NaN,NaN,NaN,NaN,NaN
16627,206997704,66965962,2021-03-05 18:08:01,NaN,NaN,NaN,NaN,NaN,NaN
16628,206997704,66994499,2021-03-05 18:10:13,NaN,NaN,NaN,NaN,NaN,NaN
16629,206997704,67981477,2021-03-05 18:08:24,NaN,NaN,NaN,NaN,NaN,NaN


filtrando somente as contribuições posteriores a 2019

In [37]:
df_first_user_commit = df_first_user_commit.loc[df_first_user_commit['created_at'] > pd.to_datetime('2019-01-01')]

NameError: name 'df_first_user_commit' is not defined

Agora calculamos quantos commits cada projeto possui no momento de onboard do dev

In [ ]:

def get_total_commits(row):
    repo_id = row['project_id']
    date = row['created_at']
    total_commits = 0
    for file in dump_open_source_files:
        chunk = pd.read_csv(dump_open_source_files)
        total_commits += chunk.loc[(chunk['project_id'] == repo_id) & (pd.to_datetime(chunk['created_at']) <= pd.to_datetime(date))].shape[0]

    return total_commits

In [ ]:

['total_commits'] = df_first_commit_user.apply(get_total_commits, axis=1)   

Pegar os detalhes do projeto

In [ ]:
df_projetos = pd.read_csv(join(path, "projects.csv"), names=["id", "url", "owner_id", "name", "descriptor", "language", "created_at", "forked_from", "deleted", "updated_at"], na_values="not available", on_bad_lines='warn', chunksize=5000000)